In [1]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import pandas as pd 
import numpy as np
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import roc_auc_score, accuracy_score, f1_score
from sklearn.model_selection import train_test_split

In [2]:
#Load the data into Pandas dataframe
df = pd.read_csv('../task2_data/task2_en_training.tsv', sep='\t')[['class', 'tweet']]
df['class'] = df['class'].apply(lambda x: int(x))
df.head()

,class,tweet
0,0,"depression hurts, cymbalta can help"
1,0,"@jessicama20045 right, but cipro can make thin..."
2,0,@fibby1123 are you on paxil .. i need help
3,0,@redicine the lamotrigine and sjs just made ch...
4,0,have decided to skip my #humira shot today. my...


In [3]:
df = df.iloc[np.random.permutation(len(df))]
df.head()

,class,tweet
19188,0,@katykateas we certainly do need to stick toge...
11145,0,but merydia
17819,0,@jimjohnsonsci rosiglitazone = *potential* wi...
10709,0,the tamiflu commercial with the mini phone is ...
3392,0,rt @mothertobaby: we've updated several of our...


https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.GradientBoostingClassifier.html

https://www.kaggle.com/beagle01/prediction-with-gradient-boosting-classifier

In [4]:
from sklearn.metrics import classification_report
from sklearn.model_selection import StratifiedKFold

X = df['tweet']
Y = df['class']

X_train, X_test, y_train, y_test = train_test_split(X, Y , test_size=0.2)

tfidf = TfidfVectorizer()
tfidf.fit(X_train)

skf = StratifiedKFold(n_splits=5)
skf.get_n_splits(X, Y)

print('Stratified K-Fold')
f1=[] # array of f1 scores 
for train_index, test_index in skf.split(X, Y):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = Y[train_index], Y[test_index]
    
    # vectorization on the each step 
    tfidf = TfidfVectorizer()
    tfidf.fit(X_train)
    X_train = tfidf.transform(X_train)
    X_test = tfidf.transform(X_test)
    
    #model
    model = GradientBoostingClassifier()
    model.fit(X_train,y_train)

    predictions_model = model.predict(X_test)
    f1.append(round(f1_score(predictions_model, y_test, average='macro')*100,3))
    print("Model F1 Score -> ", f1[-1])

# avg f1
print(f'F1 average: {round(np.array(f1).mean(), 3)} %')
print()

Stratified K-Fold
Model F1 Score ->  56.014
Model F1 Score ->  59.696
Model F1 Score ->  59.176
Model F1 Score ->  61.119
Model F1 Score ->  50.84
F1 average: 57.369 %



In [5]:
# # cross_val_score
# from sklearn.model_selection import ShuffleSplit, cross_val_score, cross_val_predict
# scores = cross_val_score(model, X_train, y_train, cv=5, scoring='f1_macro')
# print('Cross val score')
# print('Model scores:', scores)
# print(f'Model scores mean: {round(scores.mean()*100, 3)} %')
# print(f'Model scores std: {round(scores.std()*100, 3)}')